In [1]:
import requests
import json

In [2]:
def extract():
    f = open('../chapter-3/tmdb.json')
    if f:
        return json.loads(f.read())

In [3]:
def reindex(analysis_settings={}, mapping_settings={}, movie_dict = {}):
    settings = {
        "settings": {
            "number_of_shards": 1,
            "index": {
                "analysis": analysis_settings,
            }
        }
    }
    
    if mapping_settings:
        settings["mappings"] = mapping_settings
    
    resp = requests.delete("http://localhost:9200/tmdb")
    resp = requests.put("http://localhost:9200/tmdb", json=settings)
    
    bulk_movies = ""
    for id, movie in movie_dict.items():
        add_cmd = {
            "index": {"_index": "tmdb", "_id": movie["id"]}
        }
        bulk_movies += json.dumps(add_cmd) + "\n" + json.dumps(movie) + "\n"
    
    resp = requests.post(
        "http://localhost:9200/_bulk",
        data=bulk_movies,
        headers={'Content-Type': 'application/json'}
    )
    return resp

In [4]:
def search(query):
    url = 'http://localhost:9200/tmdb/_doc/_search'
    resp = requests.get(url, json=query)
    
    search_hits = json.loads(resp.text)['hits']
    
    print("Num\tRelevance\tMovie title")
    
    for idx, hit in enumerate(search_hits['hits']):
        print(f"{idx}\t{hit['_score']}\t{hit['_source']['title']}")

In [5]:
movie_dict = extract()

In [6]:
analysis = {
    "analyzer": {
        "default": {
            "type": "english"
        }
    }
}

In [7]:
reindex(analysis_settings=analysis, mapping_settings=None, movie_dict=movie_dict)

<Response [200]>

In [8]:
user_search = "basketball with cartoon aliens"

In [9]:
query = {
    "query": {
        "multi_match": {
            "query": user_search,
            "fields": ["title^0.1", "overview"]
        }
    }
}

In [10]:
search(query)

Num	Relevance	Movie title
0	5.854866	Aliens vs Predator: Requiem
1	5.104813	High School Musical
2	4.918132	Independence Day
3	4.408952	Star Trek IV: The Voyage Home
4	4.34342	Napoleon Dynamite
5	3.3585892	Predator 2
6	3.3585892	Alien
7	3.2311735	Predator
8	3.1130729	Men in Black
9	2.7159886	Men in Black II
